# Problemas para a aula 6

Este notebook apresenta algumas das soluções dos problemas propostos
<!-- TEASER_END -->

<!-- TEASER_END -->

In [ ]:
using PyPlot
using Jacobi

# Problemas

## Problema 1

Faça uma função para calcular a integral usando a regra do ponto médio com pontos cujo espaçamento não é igual


### Solução

Na regra do ponto médio, a função é calculada no meio do intervalo. Neste caso, o valor da função no ponto médio não está definido. Então este valor teria que ser interpolado. Usando uma interpolação linear, obtemos o método do trapézio!!! Por isso que o método do trapézio tem precisão pior que o método do ponto médio.


## Problema 2
Faça uma função para calcular a integral usando a regra do trapézio com pontos cujo espaçamento não é igual


In [ ]:
function trapezio(x, y::AbstractVector{T}) where {T}
    
    A = zero(T)
    n = length(y)
    
    for i = 2:n
        δx =  x[i] - x[i-1]
        yₘ = (y[i] + y[i-1]) / 2
        δA = δx * yₘ
        A += δA
    end
        
    return A
end


In [ ]:
# Exemplo: integral de 0 a π de sin(x)

# 1. Solução exata:
Ie = -(cos(π) - cos(0))
randunif(n, a, b) = rand(n) .* (b-a) .+ a
function traprand(a, b, f, n)
    
    x = sort([a; randunif(n-2, a, b); b])
    y = f.(x)
    
    return trapezio(x, y)
end
# Gerar pontos aleatórios:
n = [3; 5; 7; 10:5:100; 110:10:500; 520:20:1000; 1050:50:3000]

Ic = traprand.(0.0, π, sin, n);
err = abs.(Ic .- Ie);

In [ ]:
tr = loglog(n, err, "bo")
text(200, 0.005, L"$\mathcal{O}(n^{-2})$")
nn1 = 50:1000
ee1 = 80 .* nn1.^(-2)
quad = plot(nn1, ee1, "r--")


## Problema 3

Calcule usando os diversos métodos apresentados a seguinte integral:

$$
\int_{-1}^1 \left(1- | x |\right) \: dx
$$




In [ ]:
function midrule(a, b, f, n)
    dx = (b-a) / n
    x = range(a, b, length=n+1)
    s = 0.0
    for i in 1:n
        s += f(x[i]+dx/2)
    end
    
    return dx * s
end

function trapezoidal(a, b, f, n)
    x = range(a, b, length=n+1)
    
    s = f(x[1])/2 + f(x[end])/2
    
    for i in 2:n
        s += f(x[i])
    end
    
    return (b-a) * s/n
end
    
# Detalhe n tem que ser divisível por 2.
function simpsonsrule(a, b, f, n)
    x = range(a, b, length=n+1)
    
    s = f(x[1]) + f(x[end])
    w, w2 = 4,2
    for i = 2:n
        s += w*f(x[i])
        w, w2 = w2, w
    end
    
    return s * (b-a)/(3n)
end

function romberg(a, b, f, n)
    R1 = zeros(n)
    R2 = zeros(n)
    h = Float64(b-a)
    R1[1] = h/2 * (f(a) + f(b))
    for i in 2:n
        s = 0.0
        for k in 1:(2^(i-2))
            s += f(a + (k-0.5)*h)
        end
        R2[1] = 0.5 * (R1[1] + h * s)
        for k in 2:i
            R2[k] = R2[k-1] + (R2[k-1] - R1[k-1]) / (4^(k-1) - 1)
        end
        for k in 1:i
            R1[k] = R2[k]
        end
        h = h/2
    end
    
    return R2[n]
    
end


### Solução

In [ ]:
n1 = 2:10#; 12:2:40; 45:5:200; 210:10:500; 520:20:1000; 1050:50:2000]
a = -1.0
b = 1.0
f(x) = 1.0 - abs(x)

Ie = 1.0

### Regra do trapézio:

In [ ]:
err1 = abs.(trapezoidal.(a, b, f, n1) .- Ie;)

In [ ]:
plot(n1, err1, "bo-")


#### Usando simetria

In [ ]:
err1b = abs.(2*trapezoidal.(0, 1, f, n1) .- Ie;)

#### Só n ímpar:


In [ ]:
n1c = 1:2:101
err1c = abs.(midrule.(a, b, f, n1c) .- Ie;);


In [ ]:
loglog(n1c, err1c, "bo")
nn1 = 20:80
ee1 = 5 .* nn1.^(-2)
quad = plot(nn1, ee1, "r--")
text(40, 0.005, L"$\mathcal{O}(n^{-2})$")


### Regra do ponto médio

In [ ]:
n2 = 1:2:101
err1 = abs.(midrule.(a, b, f, n2) .- Ie;)

### Romberg

In [ ]:
n = 1:10
err3 = abs.(romberg.(-1,1,f,n) .- Ie)

### Método de Gauss

In [ ]:
n4a = 1:50
z4a = zgj.(n4a);
w4a = wgj.(z4a);
integr(z,w,f) = sum(f.(z) .* w)

err4a = abs.(integr.(z4a, w4a, f) .- Ie);

In [ ]:
loglog(n4a, err4a, "bo-")

#### Usando a simetria

In [ ]:
ξ2z(ξ) = (ξ+1.0)/2
z4b = [ξ2z.(z) for z in z4a];
err4b = abs.(integr.(z4b, w4a, f) .- Ie)

In [ ]:
#### Gauss-Lobatto-Jacobi
n4c = 2:50
z4c = zglj.(n4c);
w4c = wglj.(z4c);


In [ ]:
err4c = abs.(integr.(z4c, w4c, f) .- Ie);
loglog(n4c, err4c, "bo-")
plot(n4a, err4a, "gs--")
nn1 = 20:80
ee1 = 5 .* nn1.^(-2)
quad = plot(nn1, ee1, "r--")
text(40, 0.005, L"$\mathcal{O}(n^{-2})$")

## Problema 4

Em Pitometria, calcular a vazão correponde a integrar o perfil de velocidades. Deste modo, 

$$
Q = \int_0^R 2\pi r\cdot u(r) \:dr
$$

Encontre os pesos e os nós de integração usando o método de Gauss (com peso). Dica: mapeie $0\le r\le R$ para uma variável $\xi$ de modo que os limites de integração sejam $-1 \le \xi \le 1$.

### Desafio

No problema anterior, foram utilzados polinômios. Tente encontrar os pesos e nós para sequências de funções do tipo

$$
1, \:\ln x, \:x, \:x^2, \:x^3, \:x^4, \:\text{etc}
$$

### Solução

Primeiro, vamos fazer o seguinte mapemanto:

$$
0 \le r \le R \quad \Longrightarrow \quad -1 \le \xi \le 1
$$

Definindo 

$$
r = \frac{1+\xi}{2}\cdot R
$$

com este mapeamento, obtemos:

$$
\delta r = \frac{R}{2} \cdot \delta\xi
$$

Com estas transformações, obtemos:

$$
Q = \int_0^R 2\pi r\cdot u(r) \:dr = 2\pi \int_{-1}^1  r(\xi) \cdot u[r(\xi)] J(\xi)\:d\xi  = \bar{V} A = \bar{V}\pi R^2\\
\bar{V}= \frac{Q}{A}=\frac{2\pi}{\pi R^2} \int_{-1}^1 \frac{1+\xi}{2}\cdot R \cdot u[r(\xi)] \cdot \frac{R}{2} d\xi = \frac{1}{2} \int_{-1}^1 u(\xi) (1+\xi)\:d\xi
$$





A velocidade média tem, então a seguinte forma:

$$
\bar{V}= \frac{Q}{A}= \frac{1}{2} \int_{-1}^1 u(\xi) (1+\xi)\:d\xi
$$

Incorporando o peso $(1+\xi)$ pode-se chegar à seguinte regra de quadratura de Gauss:

$$
\bar{V} = \frac{1}{2} \sum_{i=1}^Q w_i^{0,1} \cdot u\left(\xi_i^{0,1}\right)
$$

onde $\xi_i^{0,\beta}$ é $w_i^{0,\beta}$ são os nós e pesos da quadratura de Gauss Jacobi com função peso $w(x) = (1+x)$.

Incorporando o fator 1/2 ao peso, e lembrando que $r = (1+\xi)/2$

In [ ]:
n = 1:28

ξ = zgj.(n, 0, 1)
w1 = wgj.(ξ, 0, 1)

r = [(1 .+ x) / 2 for x in ξ];
w = [w ./ 2 for w in w1];

In [ ]:
r[1:5]

In [ ]:
w[1:5]

#### Observação:

Uma condição que os pesos devem observar é:

$$
\sum_{i=1}^Q w_i \equiv 1
$$

In [ ]:
sum.(w)

#### Exemplo:

$$
U(r) = r^{\frac{1}{7}}
$$

Se a tubulação tiver 2 m de diâmetro, qual a velocidade média?

**Solução exata**:
$$
\bar{V} = \frac{2\pi}{\pi R^2}\int_0^R U(r) r \:dr
$$
ou seja
$$
\bar{V} = 2\int_0^1 r\cdot r^{\frac{1}{7}} \:dr = \frac{14}{15}
$$

**Solução com quadratura de Gauss Jacobi**


In [ ]:
nq = length(w)
u(r) = r^(1/7)
V = integr.(r, w, u)
errv = abs.(V .- 14/15);

In [ ]:
loglog(n, errv, "bo-")

#### Incluindo o ponto central

Durante a pitometria, é comum incluir o ponto central pois esta medição pode ser utilizada para se fazer a monitoração da vazão. Os pontos da quadratura utilizada a acima não incluem o centro. Assim, uma medição extra é necessária. Como a medição é feita ao longo de 1 diâmetro, na ida e vola, 2 pontos a mais são medidos. Incluindo este ponto na regra de quadratura, haverá uma economia de medição:

Número de medições por eixo:

 * Hoje: $4\times Q + 2$
 * Incluindo o centro: $4 \times (Q-1) + 2$
 
 Economia de 4 pontos na medição. Debaixo do sol/chuva/vento e em posição de stress isso pode fazer uma grande diferença.

In [ ]:
nr = 1:28

ξr = zgrjp.(nr, 0, 1)
w1r = wgrjp.(ξr, 0, 1)

rr = [(1 .+ x) / 2 for x in ξr];
wr = [w ./ 2 for w in w1r];

In [ ]:
Vr = integr.(rr, wr, u)
errvr = abs.(Vr .- 14/15);

In [ ]:
gj = loglog(n, errv, "bo:", label="Gauss-Jacobi")
grj = plot(nr, errvr, "rs-", label="Gauss-Radau-Jacobi")
legend()

In [ ]:
semilogy(n, errvr./errv, "bo--")

# Desafio

In [ ]:
F

In [ ]:
f!(F, [0.3, 0.6, 0.25, 0.25])

In [ ]:
sys(F, [0.3, 0.6, 0.25, 0.25])
F

In [ ]:
F = zeros(4)

In [ ]:
pitotquadw(3)


In [ ]:
log10

In [ ]:
function makesyswfun(Q)
    w = 0.5 / Q # Pesos iguais!
    
    function sys!(F, x)
        F[1] = sum(w .* log.(x)) + 0.25
        for i in 2:Q
            F[i] = sum((x.^(i-1)) .* w) - 1/(i+1)
        end
    end
    return sys!
    
end
function pitotquadw(Q)
    sys = makesyswfun(Q)
    dr = 1/(5Q)
    
    xinit = 1.0 .- 10.0.^range(log10(dr),0,length=Q+2)
    println(xinit[2:end-1])
    sol = nlsolve(sys, xinit[2:end-1], autodiff = :forward)
    
    return sol
end


In [ ]:
rr, ww = pitotquad1(5)



In [ ]:
function pitotquad1(Q)
    sys = makesysfun(Q)
    initvals = makeinitvals(Q)
    sol = nlsolve(sys, initvals, autodiff = :forward)
    
    r = sol.zero[1:Q]
    w = sol.zero[Q+1:end]
    
    return r, w
end

In [ ]:
makeinitvals(2)

In [ ]:
function makesysfun(Q)
    
    n = 2Q
    function sys!(F, x)
        r = x[1:Q]
        w = x[(Q+1):end]
        F[1] = sum(w) - 0.5
        F[2] = sum(w .* log.(r)) + 0.25
        for i in 3:n
            F[i] = sum((r.^(i-2)) .* w) - 1/i
        end
    end
    return sys!
end
#function makejacfun(Q)
#    n = 2Q
#    function jac!(F, x)
function makeinitvals(Q)
    x = range(0, 1, length=Q+2)[2:end-1]
    w = collect(range(0, 0.5, length=Q+1)[2:end])
    w .= w / (2*sum(w))
    #w = ones(Q) / (2Q)
    return [x; w]
end

    

In [ ]:
sum(w)

In [ ]:
w

In [ ]:
integrf(a)

In [ ]:
sum(w .* uu.(rz, Ref(a)))

In [ ]:
a = [0.9033; 0.0855; 0.1518; -0.05672]
uu(r,a) = a[1] + a[2]*log(1-r) + r*(a[3] + a[4]*r)


In [ ]:
w

In [ ]:
sol.zero
w = sol.zero[3:4]
rz = sol.zero[1:2]


In [ ]:
sol = nlsolve(f!, [0.3; 0.9; 0.25; 0.25], autodiff = :forward)

In [ ]:
log(1.0 - 0.999999999999)

In [ ]:
plot(r1, u1, "b-")
plot(r1, profile.(r1, Ref(a6)), "r-")

In [ ]:
a1 = ajusteperfil(r1, u1, 1)
a2 = ajusteperfil(r1, u1, 2)
a3 = ajusteperfil(r1, u1, 3)
a4 = ajusteperfil(r1, u1, 4)
a5 = ajusteperfil(r1, u1, 5)
a6 = ajusteperfil(r1, u1, 6)

In [ ]:
r1 = collect(range(0, 1, length=101)[2:100])
u1 = r1 .^ (1/7);

In [ ]:
function ajusteperfil(r, u, n)
    
    m = length(r)
    ϕ = zeros(m, n)
    ϕ[:,1] .= 1.0
    if n>1
        ϕ[:,2] .= log.(1 .- r)
        for i in 3:n
            ϕ[:,i] .= r .^ (i-2)
        end
    end
    coefs = CurveFit.fit_linear_model(ϕ, u)
    
    return coefs
end

function profile(r, a)
    n = length(a)
    u = a[1] 
    if n > 1
        u += a[2] * log(1.0-r)
        if n > 2
            uu = r*a[end]
            for i in (n-1):-1:3
                uu = r * (a[i] + uu)
            end
            u += uu
        end
    end
    
    return u
end

function vmedperfil(a)
    
    n = length(a)
    
    vm = a[1]
    if n > 1
        vm += -3*a[2]/2
        for i in 3:n
            vm += 2/i
        end
    end
    
    return vm
end


In [ ]:
function f!(F, x)
    F[1] = x[3] + x[4] - 0.5
    F[2] = x[3]*log(1-x[1]) + x[4]*log(1-x[2]) + 0.75
    F[3] = x[3]*x[1] + x[4]*x[2] - 1/3
    F[4] = x[3]*x[1]^2 + x[4]*x[2]^2 - 0.25
end

integrf(a) = a[1]/2 - a[2]*0.75 + a[3]/3 + a[4]/4


In [ ]:
using NLsolve, CurveFit


In [ ]:
ϕ, u2 = ajusteperfil(r1, u1, 3)


In [ ]:
0.25 + 0.25^2 + 0.25^3

In [ ]:
profile(0.25, b)

In [ ]:
b = [0.0, 0.0, 1, 1, 1]